## Skincare website scraper using selenium

In [1]:
import pandas as pd
from selenium import webdriver
from lxml import etree
import time
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from IPython.core.display import display, HTML

In [137]:
options = webdriver.ChromeOptions()
options.binary_location = "C:/Program Files (x86)/Google/Chrome/Application/chrome.exe"

In [211]:
#Define product list that you want to input into website search
product_list = pd.read_csv('data/skincare_list.csv')
product_list

,Input
0,moisturiser
1,masks & exfoliators
2,serums
3,cleansers
4,eye cream
5,eye serum
6,acne & blemish
7,night time
8,skin care tools
9,toners & mists


### First scraper takes pre-defined list to input into website, scrape main page for product + brand

In [139]:
driver = webdriver.Chrome(executable_path='C:/Users/csuar/Documents/chromedriver/chromedriver.exe',options=options)

In [140]:
product_types = []
product_names = []
brand_names = []

In [141]:
for item in product_list['Input'][:2]:
    try:
        #
        url = 'https://www.cultbeauty.co.uk/'
        driver.get(url)
        time.sleep(2)

        input_box=driver.find_element_by_xpath('//input[@class="js-search-input"]')
        input_box.clear()
        input_box.send_keys(item)
        time.sleep(2)

        driver.find_element_by_xpath('//input[@class="js-search-input"]').send_keys(Keys.RETURN)

        time.sleep(2)

        html = etree.HTML(driver.page_source)

        product_name = html.xpath('//h3[@class="productGridTitle"]//text()')
        brand_name = html.xpath('//div[@class="productGridBrandTitle"]//span//text()')
        
        for i in product_name:
            product_types.append(item)
            
        product_names.extend(product_name)
        brand_names.extend(brand_name)
        
        time.sleep(2)

        if len(product_names) % 1 == 0:
            listing = pd.DataFrame({'Product':product_names,'Brand':brand_names,'Type':product_types})
        
    except:
        None

In [290]:
listing.head()

,Product,Brand,Type,product+brand,Input
0,"Vitamin B, C and E Moisturizer",THE INKEY LIST,moisturiser,"Vitamin B, C and E Moisturizer, THE INKEY LIST","Vitamin B, C and E Moisturizer, THE INKEY LIST"
1,Dew Point Moisturizing Gel-Cream,Versed,moisturiser,"Dew Point Moisturizing Gel-Cream, Versed","Dew Point Moisturizing Gel-Cream, Versed"
2,Rose Deep Hydration Face Cream,fresh,moisturiser,"Rose Deep Hydration Face Cream, fresh","Rose Deep Hydration Face Cream, fresh"
3,Natural Moisturizing Factors + HA,The Ordinary,moisturiser,"Natural Moisturizing Factors + HA, The Ordinary","Natural Moisturizing Factors + HA, The Ordinary"
4,Peptide Moisturiser,THE INKEY LIST,moisturiser,"Peptide Moisturiser, THE INKEY LIST","Peptide Moisturiser, THE INKEY LIST"


In [183]:
listing['Input'] = listing['Product'].astype('str') + ", " + listing['Brand'].astype('str')

In [212]:
len(listing)

2133

In [289]:
#Duplicates located in list, drop these

In [203]:
listing1 = listing.drop_duplicates(subset=['Input'])

In [213]:
len(listing1)

1779

### Using scraped initial list, scrape individual products with details from individual product page

In [129]:
driver = webdriver.Chrome(executable_path='C:/Users/csuar/Documents/chromedriver/chromedriver.exe',options=options)

In [172]:
input_products = []
descriptions = []
how_to_uses = []
ingredients = []
reviews = []
ratings = []
images = []
brand_names = []
item_names = []

In [173]:
for item in listing['product+brand'][:50]:
    try:
        url = 'https://www.cultbeauty.co.uk/'
        driver.get(url)
        time.sleep(2)

        input_box=driver.find_element_by_xpath('//input[@class="js-search-input"]')
        input_box.clear()
        input_box.send_keys(item)
        time.sleep(2)

        driver.find_element_by_xpath('//input[@class="js-search-input"]').send_keys(Keys.RETURN)

        time.sleep(2)

        product_click=driver.find_element_by_xpath('//div[@data-rk="0"]')
        product_click.click()
        time.sleep(2)

        html = etree.HTML(driver.page_source)

        open_description=driver.find_element_by_xpath('//span[@data-name="description"]')
        open_description.click()
        time.sleep(2)

        open_how_to_use=driver.find_element_by_xpath('//span[@data-name="how-to-use"]')
        open_description.click()
        time.sleep(2)

        open_ingredients=driver.find_element_by_xpath('//span[@data-name="ingredients"]')
        open_description.click()
        time.sleep(2)

        item_name = html.xpath('//div[@class="productTitle"]//text()')
        brand_name = html.xpath('//div[@class="productBrandTitle"]//text()')
        description = html.xpath('//li[2]//div[2]//text()')
        how_to_use = html.xpath('//li[3]//div[2]//text()')
        ingredient = html.xpath('//li[4]//div[2]//text()')
        review = html.xpath('//div[@class="reviewCount"]//a//text()')[0]
        image = driver.find_element_by_xpath('//img[@class="js-big-image"]').get_attribute("src")

        time.sleep(2)

        input_products.append(item)
        
        if len(item_name)<1:
            item_names.append('blank')
        else:
            item_names.append(item_name)

        if len(brand_name)<1:
            brand_names.append('blank')
        else:
            brand_names.append(brand_name)
    
        if len(description)<1:
            descriptions.append('blank')
        else:
            descriptions.append(description)
    
        if len(how_to_use)<1:
            how_to_uses.append('blank')
        else:
            how_to_uses.append(how_to_use)
    
        if len(ingredient)<1:
            ingredients.append('blank')
        else:
            ingredients.append(ingredient)

        if len(image)<1:
            images.append('blank')
        else:
            images.append(image)
    
        if len(review)<1:
            reviews.append('blank')
        else:
            reviews.append(review)
        
        time.sleep(1)
        
    except:
        None

In [174]:
final_list = pd.DataFrame({'Input':input_products,'Product':item_names,'Brand':brand_names,
                           'Description':descriptions,'How to use':how_to_uses,
                          'Ingredients':ingredients,'Reviews':reviews,'Image':images})

In [197]:
final_list.head()

,Input,Product,Brand,Description,How to use,Ingredients,Reviews,Image
0,"Blue Light Protection and Hydration Mist, One ...",[Blue Light Protection and Hydration Mist],[One Ocean Beauty],[An innovative must-have for the 21st century ...,[Close the eyes and spritz the face 3 times a ...,"[\n , \n ...",Write a Review,https://cdn.cultbeauty.co.uk/resized/680x854/s...
1,Aloe + Immortelle Hydra-Repair Treatment Mist ...,[Aloe + Immortelle Hydra-Repair Treatment Mist ],[Odacite],[An effective and innovative way to infuse ski...,"[Use before applying your oil, serum or moistu...","[\n , \n ...",4,https://cdn.cultbeauty.co.uk/resized/680x854/s...
2,"Rose + Neroli Hydra-Vitalizing Treatment Mist,...",[Rose + Neroli Hydra-Vitalizing Treatment Mist],[Odacite],"[If your skin’s a bit confused, congested or l...",[Use as a toner after cleansing your skin or a...,"[\n , \n ...",5,https://cdn.cultbeauty.co.uk/resized/680x854/s...
3,Watermelon Glow PHA + BHA Pore-Tight Toner Hom...,[Banana Soufflé Moisture Cream Home & Away Kit],[Glow Recipe],[Boasting a 10% saving and housing a full size...,"[After serum, smooth a dime-sized amount onto ...","[\n , \n ...",1,https://cdn.cultbeauty.co.uk/resized/680x854/s...
4,"No Bad Days, Saturday Skin",[No Bad Days],[Saturday Skin],[Prepare yourself for the dreaded ‘winter skin...,"[Rub-A-Dub Refining Peel Gel, Cleanse and then...","[\n , \n ...",10,https://cdn.cultbeauty.co.uk/resized/680x854/s...


### Cleaning of final data frame

In [179]:
frames = [cb_scrape_1,cb_scrape_2,cb_scrape_3,cb_scrape_4]

In [180]:
final_df = pd.concat(frames)

In [182]:
len(final_df)

2034

In [188]:
len(listing)

2133

In [245]:
export_list = pd.merge(listing1,final_df,on='Input',how='left',)

In [206]:
len(export_list)

2111

In [246]:
export_list1 = export_list.drop_duplicates(subset=['Input'])

In [208]:
len(export_list1)

1779

In [283]:
export_list1.to_csv('Final_List_CB.csv')

In [ ]:
export_list1['Ingredients_formatted'] = export_list1['Ingredients'].str.split("',").str[3:]

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'